# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [31]:
problem = '''We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better. 

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions. 

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. 
'''

## 2. Generate the mathematical model

In [57]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [58]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Radiotherapy Treatment Planning

### Sets and Indices:

*   **I:** Set of patients, indexed by i (i = 1, 2, ..., n) where n = 17
*   **P:** Set of possible proton fractions, indexed by p (p = 0, 1, ..., 15)

### Parameters:

*   **C:** Total capacity of proton fractions (C = 100)
*   **BED<sub>i</sub>(p, 15-p):** Biological Equivalent Dose score for patient i receiving p proton fractions and (15-p) photon fractions (obtained from "ProblemData.csv")

### Decision Variables:

*   **x<sub>ip</sub>:** Binary variable equal to 1 if patient i receives p proton fractions, and 0 otherwise.

### Objective Function:

Maximize the total BED score for all patients:

```
Maximize Z = ∑_{i∈I} ∑_{p∈P} BED_i(p, 15-p) * x_ip
```

### Constraints:

1.  **Proton capacity constraint:** The total number of proton fractions assigned cannot exceed the available capacity.

```
∑_{i∈I} ∑_{p∈P} p * x_ip ≤ C
```

2.  **Treatment assignment constraint:** Each patient must receive exactly 15 fractions, which can be a combination of proton and photon fractions.

```
∑_{p∈P} x_ip = 1, ∀i∈I
```

3.  **Binary variable constraint:** The decision variables must be binary.

```
x_ip ∈ {0, 1}, ∀i∈I, p∈P
```

### Model Interpretation:

This integer linear programming model aims to maximize the total BED score for all patients while respecting the limited capacity of proton therapy. The objective function sums the product of BED scores and corresponding decision variables for all patients and possible proton fraction combinations. The constraints ensure that the total proton fractions used do not exceed the available capacity and that each patient receives exactly 15 fractions. The binary nature of the decision variables ensures that a patient either receives a specific number of proton fractions or not.

### Solving the Model:

This model can be solved using various optimization software packages such as CPLEX, Gurobi, or SCIP. The solution will provide the optimal values for the decision variables x<sub>ip</sub>, indicating the number of proton fractions each patient should receive to maximize the total BED score while respecting the capacity constraint. 


## 3. Generate the pyomo code

In [59]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [60]:
#Showing the code in a formatted way
Markdown(response2.text)

## Pyomo Implementation of Radiotherapy Treatment Planning Model

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from 'ProblemData.csv')
BED = {
    (1, 0, 15): 60, (1, 1, 14): 63,  # Sample BED scores for patient 1
    (2, 2, 13): 75, (2, 3, 12): 77,  # Sample BED scores for patient 2
    # ... (Add BED scores for all patients and fraction combinations)
}

# Set Parameters
n = 17  # Number of patients
C = 100  # Proton capacity
P = range(16)  # Possible proton fractions (0 to 15)

# Create Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.Set(initialize=range(1, n + 1))
model.P = pyo.Set(initialize=P)

# Define parameters
model.BED = pyo.Param(model.I, model.P, model.P, initialize=BED)
model.C = pyo.Param(initialize=C)

# Define decision variables
model.x = pyo.Var(model.I, model.P, domain=pyo.Binary)

# Define objective function
def objective_rule(model):
    return sum(model.BED[i, p, 15-p] * model.x[i, p] for i in model.I for p in model.P)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def proton_capacity_rule(model):
    return sum(p * model.x[i, p] for i in model.I for p in model.P) <= model.C

model.proton_capacity = pyo.Constraint(rule=proton_capacity_rule)

def treatment_assignment_rule(model, i):
    return sum(model.x[i, p] for p in model.P) == 1

model.treatment_assignment = pyo.Constraint(model.I, rule=treatment_assignment_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')  # You can use other solvers like 'gurobi' or 'cplex'
results = solver.solve(model)

# Print results (add more detailed analysis as needed)
if results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Optimal solution found.")
    for i in model.I:
        for p in model.P:
            if pyo.value(model.x[i, p]) > 0:
                print(f"Patient {i} receives {p} proton fractions.")
else:
    print("Solver could not find an optimal solution.")
```

**Explanation:**

1. **Import Pyomo:** Import the necessary Pyomo library.
2. **Sample Data:** Replace the sample `BED` dictionary with your actual data from "ProblemData.csv".
3. **Set Parameters:** Define the parameters `n` (number of patients), `C` (proton capacity), and `P` (possible proton fractions).
4. **Create Pyomo Model:** Create a `ConcreteModel` instance.
5. **Define Sets:** Define sets `I` for patients and `P` for proton fractions using `pyo.Set`.
6. **Define Parameters:** Define parameters `BED` (biological equivalent dose) and `C` (capacity) using `pyo.Param`. Note that `BED` is indexed by patient, proton fraction, and remaining photon fraction. 
7. **Define Decision Variables:** Define binary decision variables `x[i, p]` representing whether patient `i` receives `p` proton fractions using `pyo.Var`.
8. **Define Objective Function:** Define the objective function to maximize the total BED score using `pyo.Objective`. 
9. **Define Constraints:**
    *   **Proton capacity constraint:** Ensure the total proton fractions assigned do not exceed the capacity using `pyo.Constraint`.
    *   **Treatment assignment constraint:** Ensure each patient receives exactly 15 fractions (proton + photon) using `pyo.Constraint`.
10. **Solve the Model:** Choose a solver (e.g., `cbc`, `gurobi`, `cplex`) and solve the model using `solver.solve(model)`.
11. **Print Results:** Analyze the results and print the optimal solution, indicating how many proton fractions each patient should receive.

**Remember to replace the sample data with your actual data and choose an appropriate solver based on your environment and problem size.** 


## 4. Input problem data and try running the generated code

In [63]:
import pyomo.environ as pyo

# # Sample Data (Replace with actual data from 'ProblemData.csv')
# BED = {
#     (1, 0, 15): 60, (1, 1, 14): 63,  # Sample BED scores for patient 1
#     (2, 2, 13): 75, (2, 3, 12): 77,  # Sample BED scores for patient 2
#     # ... (Add BED scores for all patients and fraction combinations)
# }

#LOADING THE DATA DONE BY HUMAN
df = pd.read_csv('ProblemData.csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j, 15-j)] = dose
#END

# Set Parameters
n = 17  # Number of patients
C = 100  # Proton capacity
P = range(16)  # Possible proton fractions (0 to 15)

# Create Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.Set(initialize=range(1, n + 1))
model.P = pyo.Set(initialize=P)

# Define parameters
model.BED = pyo.Param(model.I, model.P, model.P, initialize=BED)
model.C = pyo.Param(initialize=C)

# Define decision variables
model.x = pyo.Var(model.I, model.P, domain=pyo.Binary)

# Define objective function
def objective_rule(model):
    return sum(model.BED[i, p, 15-p] * model.x[i, p] for i in model.I for p in model.P)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
def proton_capacity_rule(model):
    return sum(p * model.x[i, p] for i in model.I for p in model.P) <= model.C

model.proton_capacity = pyo.Constraint(rule=proton_capacity_rule)

def treatment_assignment_rule(model, i):
    return sum(model.x[i, p] for p in model.P) == 1

model.treatment_assignment = pyo.Constraint(model.I, rule=treatment_assignment_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi' or 'cplex'
results = solver.solve(model)

# Print results (add more detailed analysis as needed)
if results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Optimal solution found.")
    for i in model.I:
        for p in model.P:
            if pyo.value(model.x[i, p]) > 0:
                print(f"Patient {i} receives {p} proton fractions.")
else:
    print("Solver could not find an optimal solution.")

Optimal solution found.
Patient 1 receives 15 proton fractions.
Patient 2 receives 8 proton fractions.
Patient 3 receives 3 proton fractions.
Patient 4 receives 0 proton fractions.
Patient 5 receives 5 proton fractions.
Patient 6 receives 0 proton fractions.
Patient 7 receives 4 proton fractions.
Patient 8 receives 13 proton fractions.
Patient 9 receives 4 proton fractions.
Patient 10 receives 5 proton fractions.
Patient 11 receives 6 proton fractions.
Patient 12 receives 0 proton fractions.
Patient 13 receives 5 proton fractions.
Patient 14 receives 0 proton fractions.
Patient 15 receives 10 proton fractions.
Patient 16 receives 10 proton fractions.
Patient 17 receives 12 proton fractions.


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [66]:
print(response.text)

## Mathematical Optimization Model for Radiotherapy Treatment Planning

### Sets and Indices:

*   **I:** Set of patients, indexed by i (i = 1, 2, ..., n) where n = 17
*   **P:** Set of possible proton fractions, indexed by p (p = 0, 1, ..., 15)

### Parameters:

*   **C:** Total capacity of proton fractions (C = 100)
*   **BED<sub>i</sub>(p, 15-p):** Biological Equivalent Dose score for patient i receiving p proton fractions and (15-p) photon fractions (obtained from "ProblemData.csv")

### Decision Variables:

*   **x<sub>ip</sub>:** Binary variable equal to 1 if patient i receives p proton fractions, and 0 otherwise.

### Objective Function:

Maximize the total BED score for all patients:

```
Maximize Z = ∑_{i∈I} ∑_{p∈P} BED_i(p, 15-p) * x_ip
```

### Constraints:

1.  **Proton capacity constraint:** The total number of proton fractions assigned cannot exceed the available capacity.

```
∑_{i∈I} ∑_{p∈P} p * x_ip ≤ C
```

2.  **Treatment assignment constraint:** Each patient must re

In [67]:
print(response2.text)

## Pyomo Implementation of Radiotherapy Treatment Planning Model

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data from 'ProblemData.csv')
BED = {
    (1, 0, 15): 60, (1, 1, 14): 63,  # Sample BED scores for patient 1
    (2, 2, 13): 75, (2, 3, 12): 77,  # Sample BED scores for patient 2
    # ... (Add BED scores for all patients and fraction combinations)
}

# Set Parameters
n = 17  # Number of patients
C = 100  # Proton capacity
P = range(16)  # Possible proton fractions (0 to 15)

# Create Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.Set(initialize=range(1, n + 1))
model.P = pyo.Set(initialize=P)

# Define parameters
model.BED = pyo.Param(model.I, model.P, model.P, initialize=BED)
model.C = pyo.Param(initialize=C)

# Define decision variables
model.x = pyo.Var(model.I, model.P, domain=pyo.Binary)

# Define objective function
def objective_rule(model):
    return sum(model.BED[i, p, 15-p] * model.x[i, p] for i in model.I for p in 